<a href="https://colab.research.google.com/github/sumanreddy11/Anomaly-Detecton/blob/main/Anomaly_detection_version1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import scipy
from scipy.io.arff import loadarff
from scipy.io import loadmat

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict

import warnings
warnings.filterwarnings('ignore')
# This is to suppress any deprecation warnings 

import os
import sys

In [ ]:
# define parameters
target_class_name = 'Class'

labels = ['Not Fraud', 'Fraud']

input_file_name = 'thyroid.mat'
raw_data_folder = '/content/sample_data/'

In [ ]:
# load data 
dict = scipy.io.loadmat(raw_data_folder + input_file_name)
print(dict)

{'__header__': b'MATLAB 5.0 MAT-file, written by Octave 3.8.0, 2014-12-05 13:11:25 UTC', '__version__': '1.0', '__globals__': [], 'X': array([[7.74193548e-01, 1.13207547e-03, 1.37571157e-01, 2.75700935e-01,
        2.95774648e-01, 2.36065574e-01],
       [2.47311828e-01, 4.71698113e-04, 2.79886148e-01, 3.29439252e-01,
        5.35211268e-01, 1.73770492e-01],
       [4.94623656e-01, 3.58490566e-03, 2.22960152e-01, 2.33644860e-01,
        5.25821596e-01, 1.24590164e-01],
       ...,
       [9.35483871e-01, 2.45283019e-02, 1.60341556e-01, 2.82710280e-01,
        3.75586854e-01, 2.00000000e-01],
       [6.77419355e-01, 1.47169811e-03, 1.90702087e-01, 2.42990654e-01,
        3.23943662e-01, 1.95081967e-01],
       [4.83870968e-01, 3.56603774e-03, 1.90702087e-01, 2.12616822e-01,
        3.38028169e-01, 1.63934426e-01]]), 'y': array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])}


In [ ]:
X_data = dict['X']
print(X_data)

[[7.74193548e-01 1.13207547e-03 1.37571157e-01 2.75700935e-01
  2.95774648e-01 2.36065574e-01]
 [2.47311828e-01 4.71698113e-04 2.79886148e-01 3.29439252e-01
  5.35211268e-01 1.73770492e-01]
 [4.94623656e-01 3.58490566e-03 2.22960152e-01 2.33644860e-01
  5.25821596e-01 1.24590164e-01]
 ...
 [9.35483871e-01 2.45283019e-02 1.60341556e-01 2.82710280e-01
  3.75586854e-01 2.00000000e-01]
 [6.77419355e-01 1.47169811e-03 1.90702087e-01 2.42990654e-01
  3.23943662e-01 1.95081967e-01]
 [4.83870968e-01 3.56603774e-03 1.90702087e-01 2.12616822e-01
  3.38028169e-01 1.63934426e-01]]


In [ ]:
y_data = dict['y']
print(y_data)

[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]


In [ ]:
X_df = pd.DataFrame.from_dict(X_data)
print(X_df)

y_df = pd.DataFrame.from_dict(y_data)
print(y_df)

             0         1         2         3         4         5
0     0.774194  0.001132  0.137571  0.275701  0.295775  0.236066
1     0.247312  0.000472  0.279886  0.329439  0.535211  0.173770
2     0.494624  0.003585  0.222960  0.233645  0.525822  0.124590
3     0.677419  0.001698  0.156546  0.175234  0.333333  0.136066
4     0.236559  0.000472  0.241935  0.320093  0.333333  0.247541
...        ...       ...       ...       ...       ...       ...
3767  0.817204  0.000113  0.190702  0.287383  0.413146  0.188525
3768  0.430108  0.002453  0.232448  0.287383  0.446009  0.175410
3769  0.935484  0.024528  0.160342  0.282710  0.375587  0.200000
3770  0.677419  0.001472  0.190702  0.242991  0.323944  0.195082
3771  0.483871  0.003566  0.190702  0.212617  0.338028  0.163934

[3772 rows x 6 columns]
        0
0     0.0
1     0.0
2     0.0
3     0.0
4     0.0
...   ...
3767  0.0
3768  0.0
3769  0.0
3770  0.0
3771  0.0

[3772 rows x 1 columns]


In [ ]:
df = pd.concat([X_df, y_df], axis=1, join='inner')
print(df)

             0         1         2         3         4         5    0
0     0.774194  0.001132  0.137571  0.275701  0.295775  0.236066  0.0
1     0.247312  0.000472  0.279886  0.329439  0.535211  0.173770  0.0
2     0.494624  0.003585  0.222960  0.233645  0.525822  0.124590  0.0
3     0.677419  0.001698  0.156546  0.175234  0.333333  0.136066  0.0
4     0.236559  0.000472  0.241935  0.320093  0.333333  0.247541  0.0
...        ...       ...       ...       ...       ...       ...  ...
3767  0.817204  0.000113  0.190702  0.287383  0.413146  0.188525  0.0
3768  0.430108  0.002453  0.232448  0.287383  0.446009  0.175410  0.0
3769  0.935484  0.024528  0.160342  0.282710  0.375587  0.200000  0.0
3770  0.677419  0.001472  0.190702  0.242991  0.323944  0.195082  0.0
3771  0.483871  0.003566  0.190702  0.212617  0.338028  0.163934  0.0

[3772 rows x 7 columns]


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3772 entries, 0 to 3771
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       3772 non-null   float64
 1   1       3772 non-null   float64
 2   2       3772 non-null   float64
 3   3       3772 non-null   float64
 4   4       3772 non-null   float64
 5   5       3772 non-null   float64
 6   0       3772 non-null   float64
dtypes: float64(7)
memory usage: 206.4 KB


In [ ]:
df.describe()

,0,1,2,3,4,5,0
count,3772.000000,3772.000000,3772.000000,3772.000000,3772.000000,3772.000000,3772.000000
mean,0.543121,0.008983,0.186826,0.248332,0.376941,0.177301,0.024655
std,0.203790,0.043978,0.070405,0.080579,0.087382,0.054907,0.155093
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.376344,0.001132,0.156546,0.203271,0.328638,0.149180,0.000000
50%,0.569892,0.003019,0.190702,0.241822,0.375587,0.173770,0.000000
75%,0.709677,0.004528,0.213472,0.282710,0.413146,0.196721,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
# change column name
df.columns.values[6] = '6'
print(df)

             0         1         2         3         4         5    6
0     0.774194  0.001132  0.137571  0.275701  0.295775  0.236066  0.0
1     0.247312  0.000472  0.279886  0.329439  0.535211  0.173770  0.0
2     0.494624  0.003585  0.222960  0.233645  0.525822  0.124590  0.0
3     0.677419  0.001698  0.156546  0.175234  0.333333  0.136066  0.0
4     0.236559  0.000472  0.241935  0.320093  0.333333  0.247541  0.0
...        ...       ...       ...       ...       ...       ...  ...
3767  0.817204  0.000113  0.190702  0.287383  0.413146  0.188525  0.0
3768  0.430108  0.002453  0.232448  0.287383  0.446009  0.175410  0.0
3769  0.935484  0.024528  0.160342  0.282710  0.375587  0.200000  0.0
3770  0.677419  0.001472  0.190702  0.242991  0.323944  0.195082  0.0
3771  0.483871  0.003566  0.190702  0.212617  0.338028  0.163934  0.0

[3772 rows x 7 columns]


In [ ]:
# export to csv
df.to_csv('/content/sample_data/thyroid.csv')